In [12]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px

In [13]:
DATA_FILE = Path("data/processed/analytics_table.parquet")
FIGS_DIR = Path("reports/figures")

In [14]:
data1 = Path("data/processed/analytics_table.parquet")
figuers1 = Path("reports/figures")
figuers1.mkdir(parents=True, exist_ok=True)


In [15]:
df = pd.read_parquet("../data/processed/analytics_table.parquet")
df.head(7)

,order_id,user_id,amount,quantity,created_at,status,status_clean,amount__isna,quantity__isna,year,month,day,hour,weekday,date_only,country,signup_date,amount_winsor,amount__is_outlier
0,A0001,0001,12.5,1,2025-12-01 10:05:00+00:00,Paid,paid,False,False,2025.0,12.0,1.0,10.0,0.0,2025-12-01,SA,2025-11-15,12.5,False
1,A0002,0002,8.0,2,2025-12-01 11:10:00+00:00,paid,paid,False,False,2025.0,12.0,1.0,11.0,0.0,2025-12-01,SA,2025-11-20,8.135,False
2,A0003,0003,<NA>,1,2025-12-02 09:00:00+00:00,Refund,refund,True,False,2025.0,12.0,2.0,9.0,1.0,2025-12-02,AE,2025-11-22,<NA>,<NA>
3,A0004,0001,25.0,<NA>,2025-12-03 14:30:00+00:00,PAID,paid,False,True,2025.0,12.0,3.0,14.0,2.0,2025-12-03,SA,2025-11-15,25.0,False
4,A0005,0004,100.0,1,NaT,paid,paid,False,False,NaN,NaN,NaN,NaN,NaN,None,SA,2025-11-25,97.75,True


In [18]:
###Task 2
data_file = Path("../data/processed/analytics_table.parquet")
df = pd.read_parquet(data_file)
print(f"Rows: {len(df)} | Columns: {len(df.columns)}")
print(df.dtypes.head(15))
missing = df.isna().sum().sort_values(ascending=False).head(10)
print(missing)

Rows: 5 | Columns: 19
order_id               string[python]
user_id                string[python]
amount                        Float64
quantity                        Int64
created_at        datetime64[ns, UTC]
status                         object
status_clean           string[python]
amount__isna                     bool
quantity__isna                   bool
year                          float64
month                         float64
day                           float64
hour                          float64
weekday                       float64
date_only                      object
dtype: object
year                  1
month                 1
amount_winsor         1
date_only             1
weekday               1
hour                  1
day                   1
amount__is_outlier    1
created_at            1
quantity              1
dtype: int64


In [ ]:
Task 3: 
EDA Questions
Questions

1-Revenue by Country: Which markets generate the most revenue and how does the Average Order Value (AOV) vary?
2-Monthly Revenue Trend: Is the business growing month-over-month, and are there any seasonal spikes?
3-Amount Distribution: What does a typical order look like after cleaning the data (winsorization)?

In [24]:
import pandas as pd
import plotly.express as px
from pathlib import Path

data_file = Path("../data/processed/analytics_table.parquet")
figures_dir = Path("../reports/figures")
figures_dir.mkdir(parents=True, exist_ok=True)


def save_figure(fig, file_path: Path, scale: int = 2):
    file_path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(file_path), scale=scale)
    print(f"✅ Saved: {file_path}")


df = pd.read_parquet(data_file)

# Task 4: Revenue by Country 
rev_by_country = (
    df.groupby("country", dropna=False)
    .agg(n=("order_id", "size"), revenue=("amount", "sum"), aov=("amount", "mean"))
    .reset_index()
    .sort_values("revenue", ascending=False)
)
fig1 = px.bar(rev_by_country, x="country", y="revenue", title="Total Revenue by Country")
save_figure(fig1, figures_dir / "revenue_by_country.png")
fig1.show()

# --- Task 5: Monthly Revenue Trend ---
trend = (
    df.groupby("month", dropna=False)
    .agg(n=("order_id", "size"), revenue=("amount", "sum"))
    .reset_index()
    .sort_values("month")
)
fig2 = px.line(trend, x="month", y="revenue", title="Revenue Over Time (Monthly)")
save_figure(fig2, figures_dir / "revenue_trend_monthly.png")
fig2.show()

# --- Task 6: Amount Distribution ---
fig3 = px.histogram(df, x="amount_winsor", nbins=30, title="Order Amount Distribution (Winsorized)")
save_figure(fig3, figures_dir / "amount_hist_winsor.png")
fig3.show()

✅ Saved: ../reports/figures/revenue_by_country.png


✅ Saved: ../reports/figures/revenue_trend_monthly.png


✅ Saved: ../reports/figures/amount_hist_winsor.png


In [26]:
import numpy as np
import pandas as pd

def bootstrap_diff_means(a, b, n_boot=2000, seed=0):
    rng = np.random.default_rng(seed)
    
    a = pd.to_numeric(a, errors="coerce").dropna().values
    b = pd.to_numeric(b, errors="coerce").dropna().values
    
    diffs = []
    for i in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())
    
    diffs = np.array(diffs)
    
    return {
        "diff_mean": a.mean() - b.mean(),
        "ci_low": np.quantile(diffs, 0.025),
        "ci_high": np.quantile(diffs, 0.975),
    }

df["is_refund"] = df["status_clean"].eq("refund").astype(int)

group_sa = df.loc[df["country"] == "SA", "is_refund"]
group_ae = df.loc[df["country"] == "AE", "is_refund"]

print(f"n_SA: {len(group_sa)} | n_AE: {len(group_ae)}")
result = bootstrap_diff_means(group_sa, group_ae)
print(result)

n_SA: 4 | n_AE: 1
{'diff_mean': np.float64(-1.0), 'ci_low': np.float64(-1.0), 'ci_high': np.float64(-1.0)}
